In [1]:
import pandas as pd
import re

# Clean Simpson dataset

In [2]:
simpson = pd.read_csv("simpsons_dataset.csv")
simpson.rename(columns={"raw_character_text":"name","spoken_words":"text"}, inplace=True)
simpson = simpson.dropna(how="any")

print(simpson.shape)
simpson.head()

(131853, 2)


,name,text
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


In [3]:
simpson.isnull().sum()

name    0
text    0
dtype: int64

In [4]:
simpson["userId"] = simpson.groupby("name").grouper.group_info[0]
simpson["textId"] = simpson.groupby("text").grouper.group_info[0]
simpson["chatId"] = 1
simpson["chat"] = "simpson"
simpson = simpson[["chatId","chat","userId","name","textId","text"]]
simpson_chat = list(simpson.to_dict("index").values())

# Clean South Park dataset

In [5]:
south = pd.read_csv("All-seasons.csv")
south = south.drop(columns={"Season", "Episode"})
south.rename(columns={"Character":"name","Line":"text"}, inplace=True)
south.text = south.text.str.replace(r"\n",'')

print(south.shape)
south.head()

(70896, 2)


,name,text
0,Stan,"You guys, you guys! Chef is going away."
1,Kyle,Going away? For how long?
2,Stan,Forever.
3,Chef,I'm sorry boys.
4,Stan,"Chef said he's been bored, so he joining a gro..."


In [6]:
south.isnull().sum()

name    0
text    0
dtype: int64

In [7]:
south["userId"] = south.groupby("name").grouper.group_info[0]
south["textId"] = south.groupby("text").grouper.group_info[0]
south["chatId"] = 0
south["chat"] = "south_park"
south = south[["chatId","chat","userId","name","textId","text"]]
south_chat = list(south.to_dict("index").values())

# Export to Mongo

In [8]:
import pymongo
import os
from dotenv import load_dotenv
from pathlib import Path  # python3 only
env_path = Path('../src') / '.env'
load_dotenv(dotenv_path=env_path)

MGURL = os.getenv("MGURL")

myclient = pymongo.MongoClient(f"{MGURL}")
db = myclient["chat"]

In [9]:
mycol = db["south_park"]
mycol.insert_many(south_chat)

In [10]:
mycol1 = db["simpson"]
mycol1.insert_many(simpson_chat)